In [2]:
import pandas as pd
import numpy as np

In [80]:
bids = pd.read_csv('../data/bids.csv')
bids_g_bidder = bids.groupby('bidder_id')
bids_g_auction = bids.groupby('auction')
# bids_g_bidder_auction = bids.groupby(['bidder_id', 'auction'])

In [87]:
print bids['bidder_id'].unique().shape[0]

6614


In [81]:
from constants import *
def generate_basic_features_per_bidder(group):
    feature = dict()
    feature['bids_cnt'] = group.shape[0]
    feature['auction_cnt'] = group['auction'].unique().shape[0]
    feature['mean_bids_per_auction'] = feature['bids_cnt'] * 1.0 / feature['auction_cnt']
    
    feature['country_cnt'] = group['country'].unique().shape[0]
    feature['device_cnt'] = group['device'].unique().shape[0]
    feature['ip_cnt'] = group['ip'].unique().shape[0]
    feature['url_cnt'] = group['url'].unique().shape[0]
    feature['merchandise_cnt'] = group['merchandise'].unique().shape[0]
    
    feature.update(dict.fromkeys(categories, 0))
    feature.update(dict.fromkeys(countries_list, 0))
    
    for cat, value in group['merchandise'].value_counts().iteritems():
        feature[cat] = value

    for country in group['country'].unique():
        feature[str(country)] = 1

    return pd.Series(feature)
    
basic_feat_per_bidder = bids_g_bidder.apply(generate_basic_features_per_bidder)
basic_feat_per_bidder.to_csv('basic_feat_per_bidder.csv')
print basic_feat_per_bidder.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6614 entries, 001068c415025a009fee375a12cff4fcnht8y to fff2c070d8200e0a09150bd81452ce29ngcnv
Columns: 218 entries, ad to zz
dtypes: float64(218)
memory usage: 11.1+ MB
None


In [82]:
columns_interested = [name for name in basic_feat_per_bidder.columns if name not in countries_list and name not in categories]
print np.min(basic_feat_per_bidder[columns_interested], axis = 0)
print
print np.max(basic_feat_per_bidder[columns_interested], axis = 0)
print
print np.min(basic_feat_per_bidder[categories], axis = 0)
print
print np.max(basic_feat_per_bidder[categories], axis = 0)

auction_cnt              1.0
bids_cnt                 1.0
country_cnt              1.0
device_cnt               1.0
ip_cnt                   1.0
mean_bids_per_auction    1.0
merchandise_cnt          1.0
url_cnt                  1.0
dtype: float64

auction_cnt                1726.000000
bids_cnt                 515033.000000
country_cnt                 179.000000
device_cnt                 2618.000000
ip_cnt                   111918.000000
mean_bids_per_auction      1327.366667
merchandise_cnt               2.000000
url_cnt                   81376.000000
dtype: float64

auto parts          0.0
books and music     0.0
clothing            0.0
computers           0.0
furniture           0.0
home goods          0.0
jewelry             0.0
mobile              0.0
office equipment    0.0
sporting goods      0.0
dtype: float64

auto parts            7344.0
books and music      16040.0
clothing              8087.0
computers            15026.0
furniture            66027.0
home goods          236

In [97]:
def max_time_idx(group):
    return group['time'].idxmax()
idx = bids_g_auction.apply(max_time_idx)
last_bids = bids.loc[idx]
def cnt_auctions_win(group):
    return group.shape[0]
auctions_win = last_bids.groupby('bidder_id').size().reset_index(name='auctions_won_cnt')
auctions_win = auctions_win.set_index('bidder_id')
print auctions_win.info()
print auctions_win.head(10)
auctions_win.to_csv('auctions_win.csv')

<class 'pandas.core.frame.DataFrame'>
Index: 1427 entries, 0053b78cde37c4384a20d2da9aa4272aym4pb to ffd62646d600b759a985d45918bd6f0431vmz
Data columns (total 1 columns):
auctions_won_cnt    1427 non-null int64
dtypes: int64(1)
memory usage: 22.3+ KB
None
                                       auctions_won_cnt
bidder_id                                              
0053b78cde37c4384a20d2da9aa4272aym4pb                 6
00b519ec8ed5e370328451379bb708a306eoj                 1
00e0f614d9dd32dd27f6080f472d2934emlos                 3
01067975436d123f717ee5aba0dd4bbfa0937                 1
0113d101ec6aabd354adac645a1ec3e82ln88                 1
01cda526658455000913950f20cf31a2q6nsf                 9
0204227c6c8042ec8d8cc1bacb0d44b1csycb                 4
022ac3a7ce986049d9a4bede83ccf9ddctvg6                 4
023d51d2bcf7d962c766ff570fa7a21ds3c96                 2
024d6a4a489d4411f795d04b6617d1f47qvrs                 5


In [85]:
print np.min(bids['time']), np.max(bids['time'])
min_time = np.min(bids['time'])
bids['time'] = bids['time'].apply(lambda x: x-min_time)
print np.min(bids['time']), np.max(bids['time'])

9631916842105263 9772885210526315
0 140968368421052


In [86]:
def generate_time_features_per_bidder(group):
    sorted_group = group.sort_values(by=['time'])
    
    time_diff = np.ediff1d(sorted_group['time'])
    if len(time_diff) == 0:
        features = {'tdiff_min':0, 
                    'tdiff_max':0,
                    'tdiff_mean':0,
                    'tdiff_std':0,
                    'tdiff_median':0,
                    'tdiff_zeros':0}
    else:
        features = {'tdiff_min':np.min(time_diff), 
                    'tdiff_max':np.max(time_diff),
                    'tdiff_mean':np.mean(time_diff),
                    'tdiff_std':np.std(time_diff),
                    'tdiff_median':np.median(time_diff),
                    'tdiff_zeros':time_diff.shape[0]-np.count_nonzero(time_diff)}
    
    return pd.Series(features)
    
time_feat_per_bidder = bids_g_bidder.apply(generate_time_features_per_bidder)
print time_feat_per_bidder.info()
print time_feat_per_bidder.head(10)
time_feat_per_bidder.to_csv('tdiff_feat_per_bidder.csv')

<class 'pandas.core.frame.DataFrame'>
Index: 6614 entries, 001068c415025a009fee375a12cff4fcnht8y to fff2c070d8200e0a09150bd81452ce29ngcnv
Data columns (total 6 columns):
tdiff_max       6614 non-null float64
tdiff_mean      6614 non-null float64
tdiff_median    6614 non-null float64
tdiff_min       6614 non-null float64
tdiff_std       6614 non-null float64
tdiff_zeros     6614 non-null float64
dtypes: float64(6)
memory usage: 361.7+ KB
None
                                          tdiff_max    tdiff_mean  \
bidder_id                                                           
001068c415025a009fee375a12cff4fcnht8y  0.000000e+00  0.000000e+00   
002d229ffb247009810828f648afc2ef593rb  1.052632e+08  1.052632e+08   
0030a2dd87ad2733e0873062e4f83954mkj86  0.000000e+00  0.000000e+00   
003180b29c6a5f8f1d84a6b7b6f7be57tjj1o  6.058642e+13  3.297784e+13   
00486a11dff552c4bd7696265724ff81yeo9v  5.094174e+13  4.018413e+12   
0051aef3fdeacdadba664b9b3b07e04e4coc6  3.792368e+12  1.635106e+11   
00

In [88]:
def generate_price_features_per_bidder(group):
    features = {'price_min':np.min(group['time']), 
                'price_max':np.max(group['time']),
                'price_mean':np.mean(group['time']),
                'price_std':np.std(group['time']),
                'price_median':np.median(group['time'])}
    return pd.Series(features)
    
price_feat_per_bidder = bids_g_bidder.apply(generate_price_features_per_bidder)
print price_feat_per_bidder.info()
print price_feat_per_bidder.head(10)
price_feat_per_bidder.to_csv('price_feat_per_bidder.csv')

<class 'pandas.core.frame.DataFrame'>
Index: 6614 entries, 001068c415025a009fee375a12cff4fcnht8y to fff2c070d8200e0a09150bd81452ce29ngcnv
Data columns (total 5 columns):
price_max       6614 non-null float64
price_mean      6614 non-null float64
price_median    6614 non-null float64
price_min       6614 non-null float64
price_std       6614 non-null float64
dtypes: float64(5)
memory usage: 310.0+ KB
None
                                          price_max    price_mean  \
bidder_id                                                           
001068c415025a009fee375a12cff4fcnht8y  7.442821e+13  7.442821e+13   
002d229ffb247009810828f648afc2ef593rb  1.348274e+14  1.348273e+14   
0030a2dd87ad2733e0873062e4f83954mkj86  7.263711e+13  7.263711e+13   
003180b29c6a5f8f1d84a6b7b6f7be57tjj1o  7.405747e+13  5.028249e+13   
00486a11dff552c4bd7696265724ff81yeo9v  7.706953e+13  2.230893e+13   
0051aef3fdeacdadba664b9b3b07e04e4coc6  1.383097e+14  1.289859e+14   
0053b78cde37c4384a20d2da9aa4272aym4pb  7

In [89]:
from collections import defaultdict
time_response = defaultdict(list)

for auction in bids_g_auction:
    auction_group = auction[1].sort_values(by=['time'])
    last_row = None
    for i, row in auction_group.iterrows():
        if last_row is None:
            last_row = row
            continue
#         try:
#             assert row['time']-last_row['time'] >= 0
#         except:
#             print row['time'], last_row['time']
#             print auction_group['time']
#             break
        time_response[row['bidder_id']].append(row['time']-last_row['time'])
        last_row = row

time_response_df = []
for bidder_id in time_response.keys():
    time_response_df.append({'bidder_id':bidder_id,
                             'response_min': np.min(time_response[bidder_id]),
                             'response_max': np.max(time_response[bidder_id]),
                             'response_mean': np.mean(time_response[bidder_id]),
                             'response_std': np.std(time_response[bidder_id]),
                             'response_median': np.median(time_response[bidder_id])})
    
time_response_df = pd.DataFrame(time_response_df)
time_response_df.set_index('bidder_id')
print time_response_df.info()
print time_response_df.head(10)
time_response_df.to_csv('time_response_feat.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6610 entries, 0 to 6609
Data columns (total 6 columns):
bidder_id          6610 non-null object
response_max       6610 non-null int64
response_mean      6610 non-null float64
response_median    6610 non-null float64
response_min       6610 non-null int64
response_std       6610 non-null float64
dtypes: float64(3), int64(2), object(1)
memory usage: 309.9+ KB
None
                               bidder_id   response_max  response_mean  \
0  560eeb674015e1284864fbdf0d0b10bbchfvp      105263158   7.894737e+07   
1  7fe504252783ec1fd2b203d3de7040e6lflsb  1383842105264   1.568362e+10   
2  c368adc1be8ffd5a95d526b24b6c2fdbg3wmi      105263158   6.315789e+07   
3  b52027a7d41c4233fa89459bff644c89asfm5      105263158   1.052632e+08   
4  d13cd9e0a7b95c0a35f3456014865230qku5q     6421052632   2.263158e+09   
5  8abc7635daf75f9fbeb63783b7b8ccd5rvuyo     2894736842   6.015038e+08   
6  3e138d46ef0c61f6ffda8682400892b68oxyv      157894737   1.578947

In [94]:
auctions_win = auctions_win.set_index('bidder_id')
auctions_win.to_csv('auctions_win.csv')
time_response_df = time_response_df.set_index('bidder_id')
time_response_df.to_csv('time_response_feat.csv')

In [68]:
auctions_win.to_csv('auctions_win.csv')
time_feat_per_bidder.to_csv('tdiff_feat_per_bidder.csv')
price_feat_per_bidder.to_csv('price_feat_per_bidder.csv')
time_response_df.to_csv('time_response_feat.csv')

In [91]:
# sanity check
print 'auctions_won_cnt', np.count_nonzero(auctions_win['auctions_won_cnt'] <= 0)

for df in [time_feat_per_bidder, price_feat_per_bidder, time_response_df]:
    print 'unique bidder', df.index.unique().shape[0]
    for column in df.columns:
        print column, np.count_nonzero(df[column] < 0)

auctions_won_cnt 0
unique bidder 6614
tdiff_max 0
tdiff_mean 0
tdiff_median 0
tdiff_min 0
tdiff_std 0
tdiff_zeros 0
unique bidder 6614
price_max 0
price_mean 0
price_median 0
price_min 0
price_std 0
unique bidder 6610
bidder_id 0
response_max 0
response_mean 0
response_median 0
response_min 0
response_std 0


In [101]:
complex_feat = pd.merge(auctions_win, price_feat_per_bidder, left_index=True, right_index=True, how='outer')
complex_feat = complex_feat.merge(time_feat_per_bidder, left_index=True, right_index=True, how='outer')
complex_feat = complex_feat.merge(time_response_df, left_index=True, right_index=True, how='outer')
complex_feat.fillna(0, inplace=True)
print complex_feat.info()
print complex_feat.head(10)
complex_feat.to_csv('complex_feat.csv')

all_feat = pd.merge(complex_feat, basic_feat_per_bidder, left_index=True, right_index=True)

train_bidders = pd.read_csv('../data/train.csv')

train_all_feat = all_feat[all_feat.index.isin(train_bidders['bidder_id'])]
test_all_feat = all_feat[np.logical_not(all_feat.index.isin(train_bidders['bidder_id']))]

print all_feat.info()
print all_feat.head(5)
print
print train_all_feat.info()
print train_all_feat.head(5)
print 
print test_all_feat.info()

all_feat.to_csv('all_feat.csv')
train_all_feat.to_csv('train_all_feat.csv')
test_all_feat.to_csv('test_all_feat.csv')

<class 'pandas.core.frame.DataFrame'>
Index: 6614 entries, 001068c415025a009fee375a12cff4fcnht8y to fff2c070d8200e0a09150bd81452ce29ngcnv
Data columns (total 17 columns):
auctions_won_cnt    6614 non-null float64
price_max           6614 non-null float64
price_mean          6614 non-null float64
price_median        6614 non-null float64
price_min           6614 non-null float64
price_std           6614 non-null float64
tdiff_max           6614 non-null float64
tdiff_mean          6614 non-null float64
tdiff_median        6614 non-null float64
tdiff_min           6614 non-null float64
tdiff_std           6614 non-null float64
tdiff_zeros         6614 non-null float64
response_max        6614 non-null float64
response_mean       6614 non-null float64
response_median     6614 non-null float64
response_min        6614 non-null float64
response_std        6614 non-null float64
dtypes: float64(17)
memory usage: 930.1+ KB
None
                                       auctions_won_cnt     price_

<class 'pandas.core.frame.DataFrame'>
Index: 6614 entries, 001068c415025a009fee375a12cff4fcnht8y to fff2c070d8200e0a09150bd81452ce29ngcnv
Columns: 235 entries, auctions_won_cnt to zz
dtypes: float64(235)
memory usage: 12.2+ MB
None
                                       auctions_won_cnt     price_max  \
bidder_id                                                               
001068c415025a009fee375a12cff4fcnht8y               0.0  7.442821e+13   
002d229ffb247009810828f648afc2ef593rb               0.0  1.348274e+14   
0030a2dd87ad2733e0873062e4f83954mkj86               0.0  7.263711e+13   
003180b29c6a5f8f1d84a6b7b6f7be57tjj1o               0.0  7.405747e+13   
00486a11dff552c4bd7696265724ff81yeo9v               0.0  7.706953e+13   

                                         price_mean  price_median  \
bidder_id                                                           
001068c415025a009fee375a12cff4fcnht8y  7.442821e+13  7.442821e+13   
002d229ffb247009810828f648afc2ef593rb  1.348273e+

In [112]:
def generate_tdiff_change_ip_features_per_bidder(group):
    group = group.sort_values(by=['time'])
    change_ip_rows = group.loc[group['ip'].shift() != group['ip']]
    change_time = np.ediff1d(change_ip_rows['time'])
    if len(change_time) == 0:
        res = 0
    else:
        res = np.median(change_time)
    return pd.Series(res)
    
tdiff_change_ip_feat_per_bidder = bids_g_bidder.apply(generate_tdiff_change_ip_features_per_bidder)
print tdiff_change_ip_feat_per_bidder.info()
print tdiff_change_ip_feat_per_bidder.head(10)
tdiff_change_ip_feat_per_bidder.to_csv('tdiff_change_ip_feat_per_bidder.csv')

<class 'pandas.core.frame.DataFrame'>
Index: 6614 entries, 001068c415025a009fee375a12cff4fcnht8y to fff2c070d8200e0a09150bd81452ce29ngcnv
Data columns (total 1 columns):
0    6614 non-null float64
dtypes: float64(1)
memory usage: 103.3+ KB
None
                                                  0
bidder_id                                          
001068c415025a009fee375a12cff4fcnht8y  0.000000e+00
002d229ffb247009810828f648afc2ef593rb  0.000000e+00
0030a2dd87ad2733e0873062e4f83954mkj86  0.000000e+00
003180b29c6a5f8f1d84a6b7b6f7be57tjj1o  3.297784e+13
00486a11dff552c4bd7696265724ff81yeo9v  1.427947e+12
0051aef3fdeacdadba664b9b3b07e04e4coc6  1.550263e+11
0053b78cde37c4384a20d2da9aa4272aym4pb  4.210526e+08
0061edfc5b07ff3d70d693883a38d370oy4fs  6.468421e+10
00862324eb508ca5202b6d4e5f1a80fc3t3lp  0.000000e+00
009479273c288b1dd096dc3087653499lrx3c  0.000000e+00


In [119]:
tdiff_change_ip_feat_per_bidder = pd.read_csv('tdiff_change_ip_feat_per_bidder.csv')
tdiff_change_ip_feat_per_bidder.info()
tdiff_change_ip_feat_per_bidder.columns = ['bidder_id', 'tdiff_ip']
tdiff_change_ip_feat_per_bidder.columns
tdiff_change_ip_feat_per_bidder.to_csv('tdiff_change_ip_feat_per_bidder.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6614 entries, 0 to 6613
Data columns (total 2 columns):
bidder_id    6614 non-null object
0            6614 non-null float64
dtypes: float64(1), object(1)
memory usage: 103.4+ KB


In [120]:
import math
# one_day = 4547368124071.8799
# startt = 9631916820392676.0
# delta = 2.16e10
# cutoff = 1.5e8 #cutoff for "small" time scales

# How to calculate time units
# start and end times for each three-day data segment
# -9631916842105263 + 9645558894736842 = 13642052631579  #week 0
# -9695580000000000 + 9709222052631578 = 13642052631578  #week 2
# -9759243157894736 + 9772885210526315 = 13642052631579  #week 4
# gap between end week 0, start week 2: 50021105263158
# gap between end week 2, start week 4: 50021105263158

def compute_entropy(array):
    entropy = np.sum(np.log(np.arange(1,array.shape[0])))
    for _, value in array.value_counts().iteritems():
        entropy -= np.sum(np.log(np.arange(1,value)))
    return entropy
def generate_entropy_features_per_bidder(group):
    feature = dict()
    feature['ip_entropy'] = compute_entropy(group['ip'])
    feature['url_entropy'] = compute_entropy(group['url'])
    return pd.Series(feature)

entropy_feat = bids_g_bidder.apply(generate_entropy_features_per_bidder)
print entropy_feat.info()
print entropy_feat.head(10)
entropy_feat.to_csv('entropy_feat_per_bidder.csv')

<class 'pandas.core.frame.DataFrame'>
Index: 6614 entries, 001068c415025a009fee375a12cff4fcnht8y to fff2c070d8200e0a09150bd81452ce29ngcnv
Data columns (total 2 columns):
ip_entropy     6614 non-null float64
url_entropy    6614 non-null float64
dtypes: float64(2)
memory usage: 155.0+ KB
None
                                         ip_entropy   url_entropy
bidder_id                                                        
001068c415025a009fee375a12cff4fcnht8y      0.000000      0.000000
002d229ffb247009810828f648afc2ef593rb      0.000000      0.000000
0030a2dd87ad2733e0873062e4f83954mkj86      0.000000      0.000000
003180b29c6a5f8f1d84a6b7b6f7be57tjj1o      0.693147      0.693147
00486a11dff552c4bd7696265724ff81yeo9v     34.552392     23.542324
0051aef3fdeacdadba664b9b3b07e04e4coc6    108.281262     38.823934
0053b78cde37c4384a20d2da9aa4272aym4pb  55626.954427  56950.537277
0061edfc5b07ff3d70d693883a38d370oy4fs    450.472687    303.746276
00862324eb508ca5202b6d4e5f1a80fc3t3lp      0.000

In [142]:
one_day = 4547368124071.8799
startt = 9631916820392676.0

new_all_feat = pd.read_csv('all_feat.csv')
new_all_feat.drop('merchandise_cnt', 1, inplace=True)
new_all_feat.drop(labels=categories, axis=1, inplace=True)
new_all_feat.drop(labels=countries_list, axis=1, inplace=True)

tdiff_ip_feat = pd.read_csv('tdiff_ip_feat_per_bidder.csv')

new_all_feat = pd.merge(new_all_feat, tdiff_ip_feat, left_on='bidder_id', right_on='bidder_id')
print new_all_feat.info()

for column in new_all_feat.columns:
    if ((column.startswith('tdiff_') or column.startswith('response_')) and not column.endswith('_zeros')) or column == 'price_std':
        new_all_feat[column] = new_all_feat[column] / one_day
    if column.startswith('price_') and not column.endswith('_std'):
        new_all_feat[column] = (new_all_feat[column] - startt) / one_day
        
new_all_feat = new_all_feat.merge(entropy_feat, left_on = 'bidder_id', right_index = True)
new_all_feat = new_all_feat.set_index('bidder_id')
new_train_all_feat = new_all_feat[new_all_feat.index.isin(train_bidders['bidder_id'])]
new_test_all_feat = new_all_feat[np.logical_not(new_all_feat.index.isin(train_bidders['bidder_id']))]

new_all_feat.to_csv('new_all_feat.csv')
new_train_all_feat.to_csv('new_train_all_feat.csv')
new_test_all_feat.to_csv('new_test_all_feat.csv')
print new_train_all_feat.info()
print new_test_all_feat.info()
print new_all_feat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6614 entries, 0 to 6613
Data columns (total 26 columns):
bidder_id                6614 non-null object
auctions_won_cnt         6614 non-null float64
price_max                6614 non-null float64
price_mean               6614 non-null float64
price_median             6614 non-null float64
price_min                6614 non-null float64
price_std                6614 non-null float64
tdiff_max                6614 non-null float64
tdiff_mean               6614 non-null float64
tdiff_median             6614 non-null float64
tdiff_min                6614 non-null float64
tdiff_std                6614 non-null float64
tdiff_zeros              6614 non-null float64
response_max             6614 non-null float64
response_mean            6614 non-null float64
response_median          6614 non-null float64
response_min             6614 non-null float64
response_std             6614 non-null float64
auction_cnt              6614 non-null float64
b

In [145]:
country_cnt_per_auction_per_bidder = pd.read_csv('country_cnt_per_auction_per_bidder.csv')
country_cnt_per_auction_per_bidder.info()
new_all_feat = new_all_feat.merge(country_cnt_per_auction_per_bidder, left_index = True, right_on = 'bidder_id')
new_train_all_feat = new_all_feat[new_all_feat.index.isin(train_bidders['bidder_id'])]
new_test_all_feat = new_all_feat[np.logical_not(new_all_feat.index.isin(train_bidders['bidder_id']))]

new_all_feat.to_csv('new_all_feat.csv')
new_train_all_feat.to_csv('new_train_all_feat.csv')
new_test_all_feat.to_csv('new_test_all_feat.csv')
print new_train_all_feat.info()
print new_test_all_feat.info()
print new_all_feat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6614 entries, 0 to 6613
Data columns (total 2 columns):
bidder_id               6614 non-null object
country_cnt_mean_auc    6614 non-null float64
dtypes: float64(1), object(1)
memory usage: 103.4+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 29 columns):
auctions_won_cnt         0 non-null float64
price_max                0 non-null float64
price_mean               0 non-null float64
price_median             0 non-null float64
price_min                0 non-null float64
price_std                0 non-null float64
tdiff_max                0 non-null float64
tdiff_mean               0 non-null float64
tdiff_median             0 non-null float64
tdiff_min                0 non-null float64
tdiff_std                0 non-null float64
tdiff_zeros              0 non-null float64
response_max             0 non-null float64
response_mean            0 non-null float64
response_median          0 non-null floa